# Optimización científica de configuraciones TP/SL con contrato runner

## Planteamiento del problema

En una estrategia de trading con múltiples contratos, se plantea la siguiente estructura:

- Se opera con 3 contratos.
- Una parte de los contratos se liquida en un take-profit (TP) fijo, como 1:2 o 1:3.
- Un contrato se deja correr sin TP fijo, conocido como "runner".
- El stop-loss (SL) es simétrico y fijo, por ejemplo, 10 USD por contrato.

La intención es evaluar matemáticamente cuál configuración maximiza el rendimiento esperado en el largo plazo.

## Supuesto inicial erróneo

En las primeras simulaciones Monte Carlo se asumió que todas las configuraciones de TP (por ejemplo, 1:2 o 1:3) tenían la misma probabilidad de éxito (win rate), lo cual es incorrecto desde el punto de vista estadístico.

## Corrección basada en teoría de caminata aleatoria

Para un modelo neutral de mercado (como un paseo aleatorio simétrico), la probabilidad de alcanzar primero el TP en vez del SL es inversamente proporcional a la distancia del TP, según la fórmula:

$$
P(\text{TP}) = \frac{SL}{TP + SL}
$$

Esto significa que:

- Un TP 1:1 tiene un 50 % de probabilidad de alcanzarse antes que el SL.
- Un TP 1:2 tiene una probabilidad de \( \frac{1}{3} \approx 33.33 \% \).
- Un TP 1:3 tiene una probabilidad de \( \frac{1}{4} = 25 \% \).

Por tanto, no se puede asumir el mismo win rate para estrategias con TPs de distinta distancia.

## Implicaciones

La esperanza matemática de cada configuración debe tener en cuenta:
1. El número de contratos asociados a cada TP.
2. La distancia de cada TP.
3. La probabilidad ajustada de alcanzar cada TP.
4. El comportamiento estadístico del contrato runner.

## Solución implementada

Se ha corregido el simulador Monte Carlo para que, en lugar de usar un win rate fijo para todas las configuraciones, cada una tenga su propia probabilidad de éxito basada en su R:R (TP) utilizando:

$$
P_{\text{éxito}} = \frac{SL}{TP + SL}
$$

Esto permite una comparación más realista entre configuraciones de TP/SL + runner.



In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider

def expected_win_rate(tp, sl=1):
    """Calcula la probabilidad de alcanzar el TP antes del SL en un modelo neutral."""
    return sl / (tp + sl)

def simulate_rr_adjusted_outcomes(configs, sl_value=10, runner_mean=0, runner_std=0, n_trades=300):
    results = {}
    for name, config in configs.items():
        outcomes = []
        for _ in range(n_trades):
            trade_pnl = 0
            for contracts, rr in config:
                if rr == 'runner':
                    runner_pnl = np.random.normal(runner_mean, runner_std)
                    trade_pnl += contracts * runner_pnl
                else:
                    win_prob = expected_win_rate(tp=rr, sl=1)
                    win = np.random.rand() < win_prob
                    if win:
                        trade_pnl += contracts * sl_value * rr
                    else:
                        trade_pnl -= contracts * sl_value
            outcomes.append(trade_pnl)
        results[name] = outcomes
    return results

def simulate_with_runner_ui(runner_mean=20, runner_std=10, n_trades=300):
    sl_value = 10  # SL fijo por contrato

    configs = {
        "2xTP(1:2)+1Runner": [(2, 2), (1, 'runner')],
        "2xTP(1:3)+1Runner": [(2, 3), (1, 'runner')],
        "1xTP(1:1)+1xTP(1:3)+1Runner": [(1, 1), (1, 3), (1, 'runner')]
    }

    results = simulate_rr_adjusted_outcomes(
        configs=configs,
        sl_value=sl_value,
        runner_mean=runner_mean,
        runner_std=runner_std,
        n_trades=n_trades
    )

    df = pd.DataFrame(results)
    display(df.head())

    plt.figure(figsize=(12, 6))
    for name, pnl in results.items():
        plt.plot(np.cumsum(pnl), label=name)
    plt.title("Monte Carlo Simulation con probabilidades realistas por TP")
    plt.xlabel("Trade #")
    plt.ylabel("Equity acumulado ($)")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

interact(
    simulate_with_runner_ui,
    runner_mean=FloatSlider(value=0, min=-30, max=100, step=1, description='Runner Mean $'),
    runner_std=FloatSlider(value=0, min=0, max=50, step=1, description='Runner Std $'),
    n_trades=IntSlider(value=300, min=50, max=1000, step=50, description='# Trades')
)



interactive(children=(FloatSlider(value=0.0, description='Runner Mean $', min=-30.0, step=1.0), FloatSlider(va…

<function __main__.simulate_with_runner_ui(runner_mean=20, runner_std=10, n_trades=300)>

#### Explicación de parámetros del contrato Runner: `Mean` y `Std`

Ajustar `Runner Mean` y `Runner Std` te permite explorar escenarios realistas y medir cómo influye la gestión discrecional del contrato runner en tu expectativa de ganancia.

**¿Qué es `Runner Mean`?**

Es la ganancia media esperada del contrato *runner* cuando el trade ha sido exitoso (es decir, no ha tocado el stop loss).

- Si `Runner Mean = 20`, significa que cada vez que el contrato runner se deja correr, en promedio aporta $20 al resultado del trade.
- Si `Runner Mean = 0`, el contrato runner termina siempre en breakeven (sin ganancia ni pérdida).

---

**¿Qué es `Runner Std` (desviación estándar)?**

Representa la variabilidad (o dispersión) de los resultados del contrato runner. Cuanto mayor sea, más impredecible es su resultado.

- Si `Runner Std = 10`, y `Mean = 20`:
  - A veces el runner gana $10.
  - A veces gana $30.
  - A veces gana $5 o incluso puede quedar ligeramente negativo.
  - Pero el promedio de muchas operaciones sigue siendo $20.

**¿Por qué es útil?**

Te permite modelar la **incertidumbre real** de dejar correr un contrato sin TP fijo.  
En la práctica, a veces ese contrato:

- No aporta nada (breakeven).
- Da grandes beneficios (movimientos explosivos).
- A veces incluso se gira y termina con pequeñas pérdidas.

**Recomendaciones de valores**

| Situación                                  | `Runner Mean` | `Runner Std` | Comentario                                 |
|-------------------------------------------|----------------|---------------|---------------------------------------------|
| Siempre cierra en breakeven               | 0              | 0             | El runner no aporta nada                    |
| Gana $20 siempre                          | 20             | 0             | Aporta fijo $20                             |
| Promedio $20, pero con variabilidad alta  | 20             | 30            | A veces gana mucho, otras veces poco        |
| Deja correr libre, a veces negativo       | 10             | 25            | Modela que a veces pierde parte del camino  |

In [24]:
# Reimportación tras reinicio de entorno
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider

def expected_win_rate(tp, sl=1):
    return sl / (tp + sl)

def simulate_rr_adjusted_outcomes(configs, sl_value=10, runner_mean=0, runner_std=0, n_trades=300):
    summary = []
    results = {}
    for name, config in configs.items():
        outcomes = []
        for _ in range(n_trades):
            trade_pnl = 0
            for contracts, rr in config:
                if rr == 'runner':
                    runner_pnl = np.random.normal(loc=runner_mean, scale=runner_std)
                    trade_pnl += contracts * runner_pnl
                else:
                    p_win = expected_win_rate(tp=rr, sl=1)
                    if np.random.rand() < p_win:
                        trade_pnl += contracts * sl_value * rr
                    else:
                        trade_pnl -= contracts * sl_value
            outcomes.append(trade_pnl)
        results[name] = outcomes

        mean_pnl = np.mean(outcomes)
        std_pnl = np.std(outcomes)
        win_rate_sim = np.mean([1 if pnl > 0 else 0 for pnl in outcomes])
        summary.append({
            "Estrategia": name,
            "Expectativa Promedio ($)": round(mean_pnl, 2),
            "Desviación Std ($)": round(std_pnl, 2),
            "Win Rate Global (%)": round(win_rate_sim * 100, 2)
        })
    return results, pd.DataFrame(summary)

def simulate_with_runner_ui(runner_mean=0, runner_std=0, n_trades=300):
    sl_value = 10

    configs = {
        "2xTP(1:2)+1Runner": [(2, 2), (1, 'runner')],
        "2xTP(1:3)+1Runner": [(2, 3), (1, 'runner')],
        "1xTP(1:1)+1xTP(1:3)+1Runner": [(1, 1), (1, 3), (1, 'runner')]
    }

    results, summary_df = simulate_rr_adjusted_outcomes(
        configs=configs,
        sl_value=sl_value,
        runner_mean=runner_mean,
        runner_std=runner_std,
        n_trades=n_trades
    )

    # Mostrar tabla resumen
    display(summary_df)

    # Gráfico de equity
    plt.figure(figsize=(12, 6))
    for name, pnl in results.items():
        plt.plot(np.cumsum(pnl), label=name)
    plt.title("Monte Carlo con probabilidad ajustada por TP + Runner estocástico")
    plt.xlabel("Trade #")
    plt.ylabel("Equity acumulado ($)")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

interact(
    simulate_with_runner_ui,
    runner_mean=FloatSlider(value=0, min=-30, max=100, step=1, description='Runner Mean $'),
    runner_std=FloatSlider(value=0, min=0, max=50, step=1, description='Runner Std $'),
    n_trades=IntSlider(value=300, min=50, max=1000, step=50, description='# Trades')
)


interactive(children=(FloatSlider(value=0.0, description='Runner Mean $', min=-30.0, step=1.0), FloatSlider(va…

<function __main__.simulate_with_runner_ui(runner_mean=0, runner_std=0, n_trades=300)>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider

def expected_win_rate(tp, sl=1):
    return sl / (tp + sl)

def simulate_equity_paths(config, sl_value, runner_mean, runner_std, n_trades, n_sims):
    all_paths = []
    for _ in range(n_sims):
        path = []
        equity = 0
        for _ in range(n_trades):
            trade_pnl = 0
            for contracts, rr in config:
                if rr == 'runner':
                    runner_pnl = np.random.normal(loc=runner_mean, scale=runner_std)
                    trade_pnl += contracts * runner_pnl
                else:
                    p_win = expected_win_rate(tp=rr, sl=1)
                    if np.random.rand() < p_win:
                        trade_pnl += contracts * sl_value * rr
                    else:
                        trade_pnl -= contracts * sl_value
            equity += trade_pnl
            path.append(equity)
        all_paths.append(path)
    return np.array(all_paths)

def plot_separate_strategies_vertical(runner_mean=0, runner_std=0, n_trades=300, n_sims=100):
    sl_value = 10
    configs = {
        "2xTP(1:2)+1Runner": [(2, 2), (1, 'runner')],
        "2xTP(1:3)+1Runner": [(2, 3), (1, 'runner')],
        "1xTP(1:1)+1xTP(1:3)+1Runner": [(1, 1), (1, 3), (1, 'runner')]
    }

    fig, axes = plt.subplots(3, 1, figsize=(10, 12), sharex=True)

    for ax, (name, config) in zip(axes, configs.items()):
        paths = simulate_equity_paths(config, sl_value, runner_mean, runner_std, n_trades, n_sims)
        mean_path = paths.mean(axis=0)

        for path in paths:
            ax.plot(path, alpha=0.26)

        ax.plot(mean_path, linewidth=2.5, label="Media", color="black")
        ax.set_title(name)
        ax.set_ylabel("Equity acumulado ($)")
        ax.grid(True)
        ax.legend()

    axes[-1].set_xlabel("Trade #")
    plt.suptitle(f"Monte Carlo: {n_sims} simulaciones por estrategia", fontsize=14)
    plt.tight_layout()
    plt.show()

interact(
    plot_separate_strategies_vertical,
    runner_mean=FloatSlider(value=0, min=-30, max=100, step=1, description='Runner Mean $'),
    runner_std=FloatSlider(value=0, min=0, max=50, step=1, description='Runner Std $'),
    n_trades=IntSlider(value=300, min=50, max=1000, step=50, description='# Trades'),
    n_sims=IntSlider(value=100, min=1, max=500, step=10, description='Monte Carlo x Estrategia')
)






interactive(children=(FloatSlider(value=0.0, description='Runner Mean $', min=-30.0, step=1.0), FloatSlider(va…

<function __main__.plot_separate_strategies_vertical(runner_mean=0, runner_std=0, n_trades=300, n_sims=100)>